In [238]:
#libraries
from bs4 import BeautifulSoup, element
import urllib
import requests
import pandas as pd
import numpy as np
import lyricsgenius
import re
import json

#NLP stuff
#Tokenization
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords #look into if I should keep this or not
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer


In [196]:
#store lyrics from song in this list, for now it will start fresh every time
lyric_list = []

In [258]:
genius = lyricsgenius.Genius("rPVs-pFT7GhBfTxhpu-ISnNCcvCsbRt8wIwhkkEXovrADgSBQfkndQW7Ge22R5Ts")
song = genius.search_song("Gloria", "Kendrick Lamar")
lyrics = song.lyrics
print(lyrics)

Searching for "Gloria" by Kendrick Lamar...
Done.
216 ContributorsTranslationsEnglishPolskiPortuguêsEspañolgloria Lyrics“gloria” is the outro of Kendrick Lamar’s sixth studio album, GNX, which features vocals from his long-time friend and collaborator, SZA. It marks the sixth collaboration between the two.

The woman Kendrick… Read More [Intro: Deyra Barrera]
Sentado, Anita y tú

[Verse 1: Kendrick Lamar]
Me and my bitch got a complicated relationship
She's so controllin', I noticed it ain't no playin' with
I met her when I was off the porch as a teenager
She said one day I would right my wrongs and see paper
We started out young, lookin' for some identity, made a thousand mistakes
But never did we lack chemistry
Wrappin' my hand 'round your waist
That was back when you did whatever I say
Dreams of sellin' narcotics, runnin' blocks, or robbin' banks
I was in love with you, didn't know what it was with you
Kiss you in back of the class just to get a buzz with you
Some niggas wanted you,

C:\Users\Monado\AppData\Roaming\Python\Python312\site-packages\lyricsgenius\genius.py:468: FutureWarning: The constructor signature will change in a future version. It will change to Song(lyrics, body) instead of Song(client, json_dict, lyrics).
  song = Song(self, song_info, lyrics)


# Cleaning Lyrics

In [259]:
#Data Cleaning/Processing

def clean_lyrics(raw_lyrics):
    #Get rid of information shoved at beginning of string leaving only the lyrics of the song
    starter = ']' #This always finds the first closed bracket which is always the closing bracket for '[Intro]'
    position = raw_lyrics.find(starter)
    lyrics = raw_lyrics[position+1:]

    # Remove section headers like [Chorus], [Verse 1], etc.
    lyrics = re.sub(r"\[.*?\]", "", lyrics)

    # Remove extra whitespace, newlines
    lyrics = re.sub(r"\n+", " ", lyrics)
    lyrics = re.sub(r"\s+", " ", lyrics)

    # Optionally remove punctuation
    lyrics = re.sub(r"[^\w\s']", "", lyrics)

    # Lowercase
    lyrics = lyrics.lower()

    return lyrics


In [260]:
cleaned_lyrics = clean_lyrics(lyrics)
print(cleaned_lyrics)

 sentado anita y tú me and my bitch got a complicated relationship she's so controllin' i noticed it ain't no playin' with i met her when i was off the porch as a teenager she said one day i would right my wrongs and see paper we started out young lookin' for some identity made a thousand mistakes but never did we lack chemistry wrappin' my hand 'round your waist that was back when you did whatever i say dreams of sellin' narcotics runnin' blocks or robbin' banks i was in love with you didn't know what it was with you kiss you in back of the class just to get a buzz with you some niggas wanted you but they weren't committed preoccupied playin' john madden and bullshittin' remember when granny died you looked at me and said baby dry your eyes depend on me as your relief let your anger be mine we was locked in ever since then so territorial even took me 'way from my friends hmm hmm gah gah gah scared of forever you know nothin' else is gon' pass i just gotta let you know woah whenever yo

# Add songs to txt file for storage

In [ ]:
#appends lyrics to txt file to store for later use with NLP model
with open("lyrics_list.txt", "a", encoding="utf-8") as file:
    file.write("\n" + cleaned_lyrics + ',' )

In [262]:
with open("lyrics_list.txt", "r", encoding="utf-8") as f:
    text = f.read()
    lyrics_list = [song for song in text.split(',') if song]

In [263]:
print(lyrics_list)
print(len(lyrics_list))

["and i wish on all the rainbows that i see i wish on all the people we see my shooter pescatarian but he eat up all the beef 21 you gave me your ass to kiss but want the all of me goddamn i stand on business dot my i's and cross my t's all i got is these lil' pictures when i think 'bout all the g's 21 memories in my head the devil talkin' to me on god i know satan down that path but god walkin' with me put your phone light in the air if you done lost a nigga 21 kept it solid ran up a bag and didn't have to cross a nigga 21 21 21 woah bulletproof my cars yeah he a homebody fuck it kill him in his yard yeah niggas go to clubs and let down they guard 21 we got choppers out until we find somewhere to park on god pussy hit the cup and nod 21 trench hoes used to curve me now they treat me like i'm god facts i know you want my spot but you don't wanna play your part pussy if it was up to you you probably let these niggas starve 21 i got rich and opened more doors than a doorman on god all th

In [266]:
print(lyrics_list[2])


 sentado anita y tú me and my bitch got a complicated relationship she's so controllin' i noticed it ain't no playin' with i met her when i was off the porch as a teenager she said one day i would right my wrongs and see paper we started out young lookin' for some identity made a thousand mistakes but never did we lack chemistry wrappin' my hand 'round your waist that was back when you did whatever i say dreams of sellin' narcotics runnin' blocks or robbin' banks i was in love with you didn't know what it was with you kiss you in back of the class just to get a buzz with you some niggas wanted you but they weren't committed preoccupied playin' john madden and bullshittin' remember when granny died you looked at me and said baby dry your eyes depend on me as your relief let your anger be mine we was locked in ever since then so territorial even took me 'way from my friends hmm hmm gah gah gah scared of forever you know nothin' else is gon' pass i just gotta let you know woah whenever y

In [207]:
with open("lyrics_list.json", "r") as f:
    data = json.load(f)  # this should be a list

In [208]:
print(data)
print(len(data))

 yeah i've been tryna call i've been on my own for long enough maybe you can show me how to love maybe i'm goin' through withdrawals you don't even have to do too much you can turn me on with just a touch baby i look around and sin city's cold and empty oh no one's around to judge me oh i can't see clearly when you're gone i said ooh i'm blinded by the lights no i can't sleep until i feel your touch i said ooh i'm drowning in the night oh when i'm like this you're the one i trust hey hey hey i'm running out of time 'cause i can see the sun light up the sky so i hit the road in overdrive baby oh the city's cold and empty oh no one's around to judge me oh i can't see clearly when you're gone i said ooh i'm blinded by the lights no i can't sleep until i feel your touch i said ooh i'm drowning in the night oh when i'm like this you're the one i trust i'm just calling back to let you know back to let you know i could never say it on the phone say it on the phone will never let you go this t

In [209]:
data.append(cleaned_lyrics)

AttributeError: 'str' object has no attribute 'append'

In [ ]:
print(data)
print(len(data))

In [ ]:
with open('lyrics_list.json','w') as f:
    json.dump(data,f)

# Tokenization/Lemmatization/Filtering stopwords

In [115]:
# Tokenize into words
words = word_tokenize(cleaned_lyrics)

# Tokenize into sentences
sentences = sent_tokenize(cleaned_lyrics) #this one doesn't do anything 

In [116]:
print(words)

['if', 'this', 'world', 'were', 'mine', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'dreams', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'enemies', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'fahfah', 'fahfahfah', 'fahfah', 'fah', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'dreams', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'enemies', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'it', "'s", 'a', 'vibe', 'do', 'your', 'dance', 'let', "'em", 'watch', 'she', 'a', 'fan', 'he', 'a', 'flop', 'they', 'just', 'wan', 'na', 

In [117]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

In [118]:
print(lemmatized_words)

['if', 'this', 'world', 'were', 'mine', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'dream', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'enemy', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'fahfah', 'fahfahfah', 'fahfah', 'fah', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'dream', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'enemy', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'it', "'s", 'a', 'vibe', 'do', 'your', 'dance', 'let', "'em", 'watch', 'she', 'a', 'fan', 'he', 'a', 'flop', 'they', 'just', 'wan', 'na', 'kumbaya',

In [119]:
#Consider if I should keep this or not. Lose of some words may affect models effectivness
stop_words = set(stopwords.words("english"))

filtered_words = [word for word in lemmatized_words if word not in stop_words]

In [120]:
print(filtered_words)

['world', 'mine', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'like', "'s", 'hot', 'world', 'wa', 'mine', "'d", 'take', 'dream', 'make', "'em", 'multiply', 'world', 'wa', 'mine', "'d", 'take', 'enemy', 'front', 'god', 'introduce', "'em", 'light', 'hit', 'strictly', 'fire', 'fahfah', 'fahfahfah', 'fahfah', 'fah', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'like', "'s", 'hot', 'world', 'wa', 'mine', "'d", 'take', 'dream', 'make', "'em", 'multiply', 'world', 'wa', 'mine', "'d", 'take', 'enemy', 'front', 'god', 'introduce', "'em", 'light', 'hit', 'strictly', 'fire', "'s", 'vibe', 'dance', 'let', "'em", 'watch', 'fan', 'flop', 'wan', 'na', 'kumbaya', 'nah', 'world', 'concrete', 'flower', 'grow', 'heartache', 'doin', "'", 'know', 'weekend', 'get', 'poppin', "'", 'low', 'better', 'day', 'comin', "'", 'sure', 'world', 'wa', 'would', "n't", 'give', 'nobody', 'sympathy', "'d", 'take', 'away', 'pain', "'d", 'give', 'everything', 'wan', 'na', 'see', 'win', 'wan', 'na', 'see', 'world

# NLP

In [182]:
lyric_list.append(cleaned_lyrics)

In [183]:
print(lyric_list)

[" if this world were mine hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire fahfah fahfahfah fahfah fah hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire it's a vibe do your dance let 'em watch she a fan he a flop they just wanna kumbaya nah in this world concrete flowers grow heartache she only doin' what she know weekends get it poppin' on the low better days comin' for sure if this world were if it was up to me i wouldn't give these nobodies no sympathy i'd take away the pain i'd give you everything i just wanna see you win wanna see if this world were mine it go in when you out ride it do it real slow slide baby

In [184]:
print(len(lyric_list))

5


In [185]:
print(lyric_list[0])

 if this world were mine hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire fahfah fahfahfah fahfah fah hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire it's a vibe do your dance let 'em watch she a fan he a flop they just wanna kumbaya nah in this world concrete flowers grow heartache she only doin' what she know weekends get it poppin' on the low better days comin' for sure if this world were if it was up to me i wouldn't give these nobodies no sympathy i'd take away the pain i'd give you everything i just wanna see you win wanna see if this world were mine it go in when you out ride it do it real slow slide baby y

In [186]:
#Sentiment Analysis
sia = SentimentIntensityAnalyzer()
scores = sia.polarity_scores(cleaned_lyrics)

print(scores)
# {'neg': 0.0, 'neu': 0.511, 'pos': 0.489, 'compound': 0.5719}

{'neg': 0.027, 'neu': 0.667, 'pos': 0.307, 'compound': 0.9988}


In [187]:
#Determine setniment score based off compount

if scores['compound'] >= 0.05:
    sentiment = 'positive'
elif scores['compound'] <= -0.05:
    sentiment = 'negative'
else:
    sentiment = 'neutral'

In [188]:
print(sentiment)

positive


In [193]:
#Identify which words are most unique
"""tfidf_matrix needs lyrics from several songs to properly score lyrics"""

# Assume `lyrics_list` is a list of cleaned lyrics from multiple songs
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(lyric_list)

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Top keywords for a song:

def get_top_keywords(tfidf_vector, feature_names, top_n=10):
    vector_dense = tfidf_vector.toarray().flatten()
    sorted_indices = np.argsort(vector_dense)[::-1]  # Descending order
    top_keywords = [(feature_names[i], vector_dense[i]) for i in sorted_indices[:top_n] if vector_dense[i] > 0]
    return top_keywords
    # sorted_indices = np.argsort(tfidf_vector.toarray()[0])[::-1]
    # top_keywords = [feature_names[i] for i in sorted_indices[:top_n]]
    # return top_keywords

In [194]:
get_top_keywords(tfidf_matrix[0], feature_names)

[('world', 0.5510086154289483),
 ('wanna', 0.22958692309539513),
 ('em', 0.22958692309539513),
 ('days', 0.1836695384763161),
 ('better', 0.1836695384763161),
 ('comin', 0.1836695384763161),
 ('turn', 0.14818342716523641),
 ('sure', 0.13775215385723708),
 ('poppin', 0.13775215385723708),
 ('low', 0.13775215385723708)]

In [191]:
print(tfidf_matrix[0])

  (0, 178)	0.04591738461907902
  (0, 198)	0.04591738461907902
  (0, 226)	0.04591738461907902
  (0, 250)	0.04591738461907902
  (0, 141)	0.04591738461907902
  (0, 196)	0.03075140762596771
  (0, 145)	0.13775215385723708
  (0, 255)	0.14818342716523641
  (0, 242)	0.043759759596395595
  (0, 266)	0.04591738461907902
  (0, 273)	0.09183476923815805
  (0, 252)	0.037045856791309104
  (0, 129)	0.04591738461907902
  (0, 142)	0.07409171358261821
  (0, 108)	0.037045856791309104
  (0, 268)	0.04591738461907902
  (0, 177)	0.04591738461907902
  (0, 236)	0.037045856791309104
  (0, 51)	0.04591738461907902
  (0, 186)	0.037045856791309104
  (0, 80)	0.04591738461907902
  (0, 223)	0.04591738461907902
  (0, 277)	0.04591738461907902
  (0, 219)	0.04591738461907902
  (0, 30)	0.04591738461907902
  :	:
  (0, 128)	0.037045856791309104
  (0, 34)	0.04591738461907902
  (0, 258)	0.04591738461907902
  (0, 64)	0.04591738461907902
  (0, 66)	0.04591738461907902
  (0, 65)	0.09183476923815805
  (0, 227)	0.09183476923815805
  (

In [192]:
print(feature_names)

['alive' 'anymore' 'attack' 'away' 'baby' 'bad' 'badly' 'bae' 'beats'
 'beginning' 'better' 'bite' 'blinded' 'bone' 'bored' 'boyfriend'
 'calling' 'cause' 'cheap' 'city' 'clearly' 'close' 'cold' 'come' 'comin'
 'concrete' 'control' 'count' 'crawl' 'creep' 'crown' 'cube' 'cuts'
 'dadada' 'dance' 'darling' 'date' 'day' 'daydreams' 'days' 'deafening'
 'deep' 'deserve' 'didn' 'doin' 'don' 'door' 'downset' 'dream' 'dreaming'
 'dreams' 'drip' 'drop' 'drowning' 'drums' 'earth' 'easy' 'em' 'end'
 'enemies' 'enjoy' 'eye' 'eyes' 'fab' 'fah' 'fahfah' 'fahfahfah' 'fan'
 'fantasy' 'faster' 'feel' 'feels' 'fine' 'fingernails' 'flop' 'flow'
 'flowers' 'fly' 'forgotten' 'friends' 'froze' 'fuck' 'fun' 'gets' 'god'
 'goes' 'goin' 'going' 'gone' 'gonna' 'good' 'got' 'gotta' 'grow' 'guitar'
 'handle' 'hard' 'having' 'head' 'hearing' 'heart' 'heartache'
 'heartbeats' 'heaven' 'hectic' 'hey' 'hit' 'hold' 'home' 'hope' 'horse'
 'hot' 'hotels' 'images' 'innocent' 'inside' 'introduce' 'isn' 'judge'
 'just' 'ke